# Transforma de Netcdf para TIF e depois plota com Leafmap - `ARQUIVOS GOES-16`

Conhecendo os dados

In [ ]:
# importa biblioteca
import leafmap
import rasterio
import rioxarray
import xarray as xr

# leitura do dado
temp_ir = rioxarray.open_rasterio('input/S10635346_202405020300.nc')
temp_ir = xr.open_dataset('input/S10635346_202405020300.nc')

# transforma de Kelvin para Celsius
temp_ir['Band1'] = (temp_ir['Band1']/100.0)-273.15

In [ ]:
# mostra o dataset
temp_ir

In [ ]:
# valor mínimo e máximo de temperatura
print( float(temp_ir['Band1'].min()), float(temp_ir['Band1'].max()) )

In [ ]:
# importa biblioteca
import leafmap
import rasterio
import rioxarray
import xarray as xr

# transforma de netcdf para tif
filename = 'S10635346_202405020300.nc'
tif = "temp_ir_ABI_GOES16.tif"
leafmap.netcdf_to_tif(f'input/{filename}', f'input/{tif}', variables=["Band1"], shift_lon=False)

# plota mapa
Map = leafmap.Map(layers_control=True)

Map.add_raster(f'input/{tif}', indexes=[1], palette="coolwarm", layer_name="Band1")

shp_estados_brasil = 'https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson'
Map.add_geojson(shp_estados_brasil, layer_name="Estados do Brasil")

Map

# Plota direto o Netcdf com Leafmap - `ARQUIVOS GOES-16`

In [ ]:
# importa biblioteca
import leafmap
import rasterio
import rioxarray
import xarray as xr

# nome do arquivo netcdf
filename = 'S10635346_202405020300.nc'

# plota mapa
Map = leafmap.Map()

Map.add_netcdf(f'input/{filename}',
               variables=["Band1"],
               palette="coolwarm",
               shift_lon=True,
               layer_name="Band1",
               indexes=[1])

shp_estados_brasil = 'https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson'
Map.add_geojson(shp_estados_brasil, layer_name="Estados do Brasil")

Map

# Plota com Leafmap usando - `ARQUIVOS GOES-16` - DOING

## Define funções

In [1]:
def download_GOES16_ch13(url, filename):
    import requests
    import warnings
    warnings.filterwarnings("ignore")
    url_filename = f'{url}{filename}'
    myfile = requests.get(url_filename, verify=False)
    open(f'{filename}', 'wb').write(myfile.content)
    return

def netcdf_to_tif(file_netcdf):
    
    # leitura do arquivo NETCDF
    nc_file = xr.open_dataset(file_netcdf)

    # extração da variável de interesse
    temp_ir = nc_file['Band1']

    # transforma de Kelvin para Celsius
    temp_ir = (temp_ir/100.0)-273.15

    # dimensões
    temp_ir = temp_ir.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
    temp_ir.rio.crs

    # define a projeção CRS
    temp_ir.rio.write_crs('epsg: 4326', inplace=True)

    # salva arquivo
    temp_ir.rio.to_raster(f'{file_tif}.tiff')

    return

## Baixa o arquivo netcdf e transforma para tif

In [2]:
# importa biblioteca
import xarray as xr
import leafmap
import rasterio
import rioxarray
import xarray as xr

# Define a data
ano, mes, dia, hor, min = '2024', '05', '02', '04', '00'

# baixando o arquivo NETCDF
url = f'https://ftp.cptec.inpe.br/goes/goes16/retangular/ch13/{ano}/{mes}/'
file_netcdf = f'S10635346_{ano}{mes}{dia}{hor}{min}.nc'
file_tif = f'S10635346_{ano}{mes}{dia}{hor}{min}'
download_GOES16_ch13(url, file_netcdf)

# transforma para TIF
netcdf_to_tif(file_netcdf)

## Plota mapa

In [9]:
# define mapa
Map = leafmap.Map(layers_control=True, zoom=5)

# shapefiles
shp_paises = "https://raw.githubusercontent.com/opengeos/leafmap/master/examples/data/countries.geojson"
Map.add_geojson(shp_paises, layer_name="Estados do Brasil", style = {"stroke": True,
                                                                             "color": 'black',
                                                                             "weight": 1.0,
                                                                             "opacity": 0.5,
                                                                             "fill": False,
                                                                             "fillColor": "#3388ff",
                                                                             "fillOpacity": 0.1})

shp_estados_brasil = 'https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson'
Map.add_geojson(shp_estados_brasil, layer_name="Estados do Brasil", style = {"stroke": True,
                                                                             "color": 'black',
                                                                             "weight": 1.7,
                                                                             "opacity": 0.5,
                                                                             "fill": False,
                                                                             "fillColor": "#3388ff",
                                                                             "fillOpacity": 0.1})
# plota mapa
vmin, vmax = -103.0, 105
Map.add_raster(f'{file_tif}.tiff', 
               indexes=[1], 
               vmin=vmin, 
               vmax=vmax,
               colormap="gray_r", 
               layer_name="Temperatura de Brilho CH13")

# plota texto 
text = f'GOES16 CH13: {ano}-{mes}-{dia} às {hor}{min} UTC'
params = {"fontsize": 20,
          "fontcolor": "blue",
          "bold": True,
          "padding": "2px",
          "background": True,
          "bg_color": "white",
          "border_radius": "5px",
          "position": "topleft"}
Map.add_text(text, **params)

# barra de cores
Map.add_colormap(cmap="gray_r",
                 discrete=True, 
                 label='Temperatura de Brilho ($^o$C)',
                 font_size=8,
                 transparent_bg=True, 
                 position='bottomright',
                 width=3, height=0.4,
                 vmin=vmin, 
                 vmax=vmax)

# exibe o mapa
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [4]:
# salva imagem para HTML
Map.to_html(f"map_goes16_{ano}{mes}{dia}_{hor}{min}.html")

In [3]:
# LEITURA DO ARQUIVO TIF GERADO
ds = rioxarray.open_rasterio(f'{file_tif}.tiff')
ds

<xarray.DataArray (band: 1, y: 3131, x: 3131)> Size: 78MB
[9803161 values with dtype=float64]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 25kB -116.0 -116.0 -115.9 ... -25.07 -25.04 -25.01
  * y            (y) float64 25kB -55.99 -55.96 -55.93 ... 34.93 34.96 34.99
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      Band1

In [4]:
import rasterio as rio
import numpy as np
with rio.open(f'{file_tif}.tiff') as tif:

    # Limites da imagem
    lonmin, latmin, lonmax, latmax = tif.bounds

    # Informações do arquivo no formato de dicionario dê um print(meta) para ver mais coisas
    meta = tif.meta

    # Resolução do arquivo
    res = tif.res

    #data = tif.indexes
    # Lê a matriz
    data = tif.read(1)

print(np.min(data), np.max(data))

-89.38999999999999 28.660000000000025


# Plota direto o Netcdf com Leafmap - `ARQUIVOS RADAR`

Conhecendo os dados


In [ ]:
import xarray as xr
radar = xr.open_dataset('input/chuva_natal_201406.nc')
radar

In [ ]:
print(float(radar['precipitacao'].min()), float(radar['precipitacao'].max()))

In [ ]:
radar['precipitacao'].plot(cmap='jet', vmin=50)

In [ ]:
# importa biblioteca
import leafmap
import rasterio
import rioxarray
import xarray as xr

# nome do arquivo netcdf
filename = 'chuva_natal_201406-8.nc'

# plota mapa
vmin, vmax = 50, 400
Map = leafmap.Map(layers_control=True, width=700, height=700, zoom=12)
Map.add_netcdf(f'input/{filename}',
               variables=["precipitacao"],
               shift_lon=False,
               layer_name="precipitacao",
               indexes=[1],
               vmin=vmin,
               vmax=vmax,
               nodata=0.0,
               palette='jet')

# barra de cores
Map.add_colormap(cmap='jet',
                 vmin=vmin, 
                 vmax=vmax,
                 discrete=True, 
                 label='Precipitação (mm)',
                 font_size=8,
                 transparent_bg=True, 
                 position='bottomright',
                 width=3, height=0.4)

# shapefile
shp_estados_brasil = 'https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson'
Map.add_geojson(shp_estados_brasil, layer_name="Estados do Brasil")

# exibe o mapa
Map